---
title: "Modeling"
format: html
editor: visual
---


## Introduction and Data

Music features and preferences vary across different genres which can lead to varying definitions of what makes a song popular in its genre. This project aims to answer the following questions:

1.  What audio features best predict a track's popularity within specific genres, and how do these success factors differ across genres?
2.  Could track lists that deviate from the typical features of their genre still be successful?

In order to predict if a song is a hit and is actually 'popular' I created a vzariable called 'is_hit' to be a benchmark for determining if a song is popular or not.

To assess the popularity of a song, we will take a look at the following music features: `jazz, country, rock, dubstep, pop, heavy-metal, bluegrass, soul, reggaeton, house, techno, k-pop`


In [ ]:
import pandas as pd 
from load_data import load_file


df = load_file()

From what we can see in each of these genres, not all songs will achieve the 'hit' status. In our code, which you will see in the coming steps, we disregard the genres where there are no hits or if all the songs have a hit status.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot popularity distribution for selected genres
interesting_genres = ['jazz', 'country', 'rock', 'dubstep', 'pop', 'heavy-metal','bluegrass', 'soul', 'reggaeton', 'house', 'techno', 'k-pop']
df_filtered = df[df['track_genre'].isin(interesting_genres)]

sns.boxplot(x='track_genre', y='popularity', data=df_filtered)
plt.xticks(rotation=45)
plt.title("Popularity Distribution by Genre")
plt.show()

## Methodology

To answer the research questions above, specifically the first one, logistic regression model is used to help classify a song as 'a_hit'(1 or 0) based on the features mentioned earlier. This model is run on each genre individually. The data set is split between training and testing with 80% set aside for training and 20% for testing. With the this model, we can take a look at the feature coefficients to see how each feature directly contributes to a songs 'hit' status.

We will train the model using these features:


In [ ]:
numeric_features = ['danceability', 'energy', 'valence', 'acousticness',
                        'speechiness', 'instrumentalness', 'tempo', 'loudness']

The code used to train the data follows these steps

1.  Filter the data frame by the specific genres selected above
2.  Split the data
3.  Standardize and use One Hot Encoding to transform genre columns
4.  Incorporate oversampling the minority class ('is_hit' == 1)
5.  Trained an Artificial Neural Network - MultiLayerPerceptron model
6.  Evaluated the test set's accuracy and f1 evaluation metrics
7.  Use Permutation Importance to calculate feature contributions in each genre


In [ ]:
# Define the function for calculating feature importance per genre and generating plots 
def feature_importance(genres, X, y, model, numeric_features):
    for genre in genres:
        genre_col = f'track_genre_{genre}'
        if genre_col not in X.columns:
            print(f"'{genre_col}' not found. Skipping genre: {genre}")
            continue

        genre_subset = X[X[genre_col] == 1]
        y_genre = y.loc[genre_subset.index]

        if genre_subset.empty:
            print(f"No samples found for genre: {genre}")
            continue

        r = permutation_importance(
            model, genre_subset, y_genre,
            n_repeats=30, random_state=0, n_jobs=-1
        )

        imp_df = pd.DataFrame({
            'feature': genre_subset.columns,
            'importance_mean': r.importances_mean,
            'importance_std': r.importances_std
        })

        imp_df_numeric = imp_df[imp_df['feature'].isin(numeric_features)].sort_values(by='importance_mean')

        plt.figure(figsize=(10, 6))
        plt.barh(imp_df_numeric['feature'], imp_df_numeric['importance_mean'],
                 xerr=imp_df_numeric['importance_std'])
        plt.xlabel("Permutation Importance")
        plt.ylabel("Audio Feature")
        plt.title(f"Feature Importance for ANN - Genre: {genre}")
        plt.tight_layout()
        plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import RandomOverSampler

# interesting genres
distinct_genres = ['jazz', 'country', 'rock', 'dubstep', 'pop', 'heavy-metal','bluegrass', 'soul', 'reggaeton', 'house', 'techno', 'k-pop']
df = df[df['track_genre'].isin(distinct_genres)].copy()


y = df['is_hit']
X_raw = df.drop(columns=['is_hit'])

X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, random_state=42)


# PREPROCESSING STEPS -> scale, encode, correct class imbalances
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw[numeric_features])
X_test_scaled = scaler.transform(X_test_raw[numeric_features])

X_train_scaled = pd.DataFrame(X_train_scaled, columns=numeric_features, index=X_train_raw.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=numeric_features, index=X_test_raw.index)


ohe = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')
train_genre_encoded = ohe.fit_transform(X_train_raw[['track_genre']])
test_genre_encoded = ohe.transform(X_test_raw[['track_genre']])

genre_cols = ohe.get_feature_names_out(['track_genre'])
train_genre_df = pd.DataFrame(train_genre_encoded, columns=genre_cols, index=X_train_raw.index)
test_genre_df = pd.DataFrame(test_genre_encoded, columns=genre_cols, index=X_test_raw.index)


X_train = pd.concat([X_train_scaled, train_genre_df], axis=1)
X_test = pd.concat([X_test_scaled, test_genre_df], axis=1)

# oversample the minority class (is hit) 
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

print("\nTraining ANN model...")
ann = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    batch_size=64,
    learning_rate_init=0.001,
    max_iter=2000,
    random_state=42
)
ann.fit(X_resampled, y_resampled)

y_pred = ann.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

feature_importance(distinct_genres, X_test, y_test, ann, numeric_features)

Since there are more samples of songs that are not hits (0.0) as compared to hits (1.0) our measurements for accuracy is very misleading. This is a common problem when it comes to working with binary classification models due to imbalances in the dataset. To account for these discrepancies we can utilize certain tools like oversampling the minority class to increase the chances of our model finding a hit song.

In order to answer the next question "Can a song that deviates from the norm of its genre still be popular", I implemented a KMeans clustering model to create natural groupings of songs and to see if there are any songs that deviate from the clusters that have higher popularity scores.


In [ ]:
from sklearn.cluster import KMeans

for genre in interesting_genres:
  num_cluster = 3
  df_genre = df[df['track_genre'] == genre].copy()
  scaled = StandardScaler()
  X_scaled = scaled.fit_transform(df_genre[numeric_features])
  kmeans = KMeans(n_clusters= num_cluster, random_state=0, n_init="auto").fit(X_scaled)
  
  df_genre['kmeans_cluster'] = kmeans.fit_predict(X_scaled)
  df_genre['kmeans_deviation'] = kmeans.transform(X_scaled).min(axis=1)
  
    
    
  plt.figure(figsize=(6, 4))
  sns.scatterplot(x='kmeans_deviation', y='popularity', data=df_genre)
  plt.title(f"{genre.title()} - Deviation vs Popularity")
  plt.xlabel("Deviation from Cluster Center")
  plt.ylabel("Popularity")
  plt.show()
  
  plt.figure(figsize=(6, 4))
  sns.kdeplot(data=df_genre, x='kmeans_deviation', hue='is_hit', common_norm=False, fill=True)
  plt.title(f"{genre.title()} – Deviation Distribution by Hit Status")
  plt.xlabel("Deviation from Cluster Center")
  plt.ylabel("Density")


## Interpretation

From what we see here, the model performs pretty well in terms of accuratley assessing if a song is not a hit. This is due to the larger volume of observations where tracks don't go over the 'is_hit' threshold of 75. However because of this, we can see our precision is lower for predicting a 'hit' song at around 0.23. Due to the low chances of songs being a hit, there are less observations of hit songs in the set. With our target class being a minority, this can lead to dataset imbalances which impact the interpretability of our model's accuracy. This is why I chose to use the f1 evaluation metric to make sure I can properly assess its precision and recall scores which gives a better understanding of the model's performance. The barplots help identify the features that contribute to a song's popularity within a genre. If a bar is greater than 0 it indicates that the feature has a positive impact to the popularity of a song and the larger the magnitude, the stronger the influence, and vice versa.

As for the second question, can songs that deviate from their genre's fingerprint still be successful, the answer to this question comes from taking a closer look at the scatter plots and smoothed histograms of each genre. From what can be seen in each of the smoothed histogram plots, there is a strong overlap between the hit and non-hit songs which can indicate that deviation from the norm doesn't matter much and doesn't impact popularity of a song or if it can be classified as a hit. Howver it is interesting to note that some of the genres have a broader distribution of deviations such as pop and country. The tails of these distributions extend towards the right which can indicate that they have some hits that are very unusual compared to the norm. Most of the genres have a common trend of having a higher density of songs 2 deviations away from the norm for both hit and non-hit categories. This can mean that both hit and non-hit songs can be defined as atypical based on how the clusters are defined. This concludes that deviating from the genre's fingerprint is a weak predictor of a successful song.

As a result, a songs success within a genre depends on a unique blend of audio features which we have demonstated earlier with the heat map of the features and their genres. Using the neural network MLP model, we see that it can perform well with the given features in a binary classification problem. Song deviations, as we discussed earlier, don't indicate much about the success of the song itself which we questioned in our research motivation. These findings help solidify our understanding of what sort of features make up a genre and which ones contribute positively and negatively. This can be a helpful tool for those who are interested in producing songs in a specific genre and want to understand what elements can help create a hit. It can also be reassuring to know that songs don't need to follow these exact feature contribution mappings to be successful given that most songs that are hits are within a certain deviation away from the genre norm.